In [ ]:
import gzip
from tqdm import tqdm
#将first_jump作为第二跳的起点，保存在second_initial中
second_initial = []
with gzip.open('first_jump.gz','rb') as f:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        second_initial.append(line.strip().decode())

#开始第二跳
str_flag="<http://rdf.freebase.com/ns/"
str_fault="<http://rdf.freebase.com/ns/people.person.date_of_birth>"
with gzip.open('freebase_douban.gz','rb') as f, gzip.open('output_second_entity.gz','wt') as output1, gzip.open('output_second_3.gz','wt') as output2:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        triplet = line.strip().decode().split('\t')[:3]
        if triplet[2] in second_initial and str_flag in triplet[0] and str_fault not in triplet[1]:
            output1.write(triplet[0]+"\n")
            output2.write(triplet[0]+" "+triplet[1]+" "+triplet[2]+"\n")
        elif triplet[0] in second_initial and str_flag in triplet[2] and str_fault not in triplet[1]:
            output1.write(triplet[2]+"\n")
            output2.write(triplet[0]+" "+triplet[1]+" "+triplet[2]+"\n")

In [1]:
import gzip
from tqdm import tqdm
min_relation = 50
min_entity = 15
max_entity = 20000

In [2]:
#将output_second_3中的内容复制到second_3.gz
with gzip.open('output_second_3.gz','rb') as f, gzip.open('second_3.gz','wt') as output:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        output.write(line.strip().decode()+"\n")
#将output_second_entity中的内容复制到second_entity.gz
with gzip.open('output_second_entity.gz','rb') as f, gzip.open('second_entity.gz','wt') as output:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        output.write(line.strip().decode()+"\n")

Processing lines: 317454704line [20:07, 262992.76line/s]
Processing lines: 317454704line [16:13, 326197.61line/s]


In [59]:
def delete_line(line_to_delete):#line_to_delete是一个list,存储了需要删除的行的行号
    with gzip.open('second_3.gz','rb') as second_3:
    #删除line_to_deiete中的对应行，保存在temp_3中
        with gzip.open('temp_3.gz','wt') as output_temp_3:
            line_num = 0
            i = 0
            for line in tqdm(second_3, desc="Processing lines", unit="line"):
                if(line_num != line_to_delete[i]):
                    output_temp_3.write(line.decode())
                else:
                    i = i + 1
                line_num = line_num + 1
    print("task1 successfully!")
    #将temp_3中的内容复制到second_3中
    with gzip.open('temp_3.gz','rb') as output_temp_3:
        with gzip.open('second_3.gz','wt') as output_second_3:
            for line in tqdm(output_temp_3, desc="Processing lines", unit="line"):
                output_second_3.write(line.decode())
    print("task2 successfully!")
    with gzip.open('second_entity.gz','rb') as second_entity:
    #删除line_to_deiete中的对应行，保存在output_second_entity中
        with gzip.open('temp_entity.gz','wt') as output_temp_entity:
            line_num = 0
            i = 0
            for line in tqdm(second_entity, desc="Processing lines", unit="line"):
                if(line_num != line_to_delete[i]):
                    output_temp_entity.write(line.decode())
                else:
                    i = i + 1
                line_num = line_num + 1
    print("task3 successfully!")
    #将output_temp_entity中的内容复制到output_second_entity中
    with gzip.open('temp_entity.gz','rb') as output_temp_entity:
        with gzip.open('second_entity.gz','wt') as output_second_entity:
            for line in tqdm(output_temp_entity, desc="Processing lines", unit="line"):
                output_second_entity.write(line.decode())
    print("delete line successfully!")
    

In [10]:
def update_second_3_by_relation():
    with gzip.open('second_3.gz','rb') as second_3:
        num_relation = {}
        for line in tqdm(second_3, desc="Processing lines", unit="line"):
            triplet = line.strip().decode().split(' ')
            if triplet[1] not in num_relation:
                num_relation[triplet[1]] = 1
            else:
                num_relation[triplet[1]] += 1
        print("num_relation",len(num_relation))
        line_to_delete = []
        line_number = 0
        second_3.seek(0)
        for line in tqdm(second_3, desc="Processing lines", unit="line"):
            triplet = line.strip().decode().split(" ")[:3]
            if num_relation[triplet[1]] <= min_relation:
                line_to_delete.append(line_number)
            line_number = line_number + 1
        print(len(line_to_delete))
        #delete_line(line_to_delete)
        return line_to_delete
#update_second_3_by_relation()

In [46]:
def update_second_3_by_entity(line_to_delete):
    with gzip.open('second_entity.gz','rb') as second_entity:
        num_entity = {}
        for line in tqdm(second_entity, desc="Processing lines", unit="line"):
            entity = line.strip().decode()
            if entity not in num_entity:
                num_entity[entity] = 1
            else:
                num_entity[entity] += 1
        print("num_entity",len(num_entity))
        line_number = 0
        second_entity.seek(0) 
        for line in tqdm(second_entity, desc="Processing lines", unit="line"):
            entity = line.strip().decode()
            if num_entity[entity] <= min_entity or num_entity[entity] >= max_entity:
                line_to_delete.append(line_number)
            line_number = line_number + 1
        print(len(line_to_delete))
        
        #delete_line(line_to_delete)
    #return line_to_delete

#update_second_3_by_entity()
    

In [47]:
#初步筛选后的second_3和second_entity保存在second_3.gz和second_entity.gz中
line_to_delete = []
update_second_3_by_entity(line_to_delete)

Processing lines: 317454704line [06:50, 774109.47line/s]


num_entity 50686843


Processing lines: 317454704line [07:47, 678478.43line/s]

308325674


In [62]:
print(line_to_delete[0])
print(len(line_to_delete))

43
308325674


In [64]:
delete_line(line_to_delete)

Processing lines: 317454704line [05:00, 1058007.06line/s]
Processing lines: 9129030line [00:30, 300403.39line/s]
Processing lines: 317454704line [04:25, 1196939.79line/s]
Processing lines: 9129030line [00:20, 439250.09line/s]

delete line successfully!
